<a href="https://colab.research.google.com/github/yodaboop/Capstone-Project/blob/main/Escalation_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---------------------------------------------------
# 1. SETUP AND DEPENDENCIES
# ---------------------------------------------------
!pip install -q langgraph langchain-core

import operator
import random
from typing import Annotated, TypedDict
from langchain_core.messages import BaseMessage, AIMessage
from langgraph.graph import StateGraph, END

# ---------------------------------------------------
# 2. DEFINE STATE AND NODE
# ---------------------------------------------------

# The state only needs to track messages now
class AgentState(TypedDict):
    messages: Annotated[list[BaseMessage], operator.add]

def create_ticket_node(state: AgentState):
    """
    This node is responsible ONLY for creating a ticket.
    It assumes the Supervisor has already decided a human is needed.
    """
    # Get the conversation history to summarize the issue
    # In a real system, you might send this full history to the API
    messages = state["messages"]
    last_user_msg = messages[-1].content

    # ---------------------------------------------------------
    # MOCK TICKET CREATION (Replace with MCP/Jira API call later)
    # ---------------------------------------------------------
    ticket_id = f"INC-{random.randint(1000, 9999)}"

    response = (
        f"✅ Support Ticket Created Successfully.\\n"
        f"🎫 **Ticket ID:** {ticket_id}\\n"
        f"📝 **Summary:** {last_user_msg[:50]}...\\n\\n"
        f"A human specialist has been notified and will contact you shortly."
    )

    return {
        "messages": [AIMessage(content=response)]
    }

# ---------------------------------------------------
# 3. BUILD THE GRAPH
# ---------------------------------------------------

workflow = StateGraph(AgentState)

# Add the single node
workflow.add_node("create_ticket", create_ticket_node)

# Entry point -> Create Ticket -> END
workflow.set_entry_point("create_ticket")
workflow.add_edge("create_ticket", END)

# Compile the graph and assign it to a global variable
# so the Supervisor notebook can import it
escalation_graph = workflow.compile()

print("✅ Escalation Agent (Ticket Creator) Ready.")

# ---------------------------------------------------
# 4. OPTIONAL: TEST IT LOCALLY
# ---------------------------------------------------
# You can run this block to verify it works before connecting to the Supervisor
from langchain_core.messages import HumanMessage

if __name__ == "__main__":
    print("\n--- Testing Escalation Agent Locally ---")
    test_input = {"messages": [HumanMessage(content="My VPN is totally broken and I need help now.")]}
    result = escalation_graph.invoke(test_input)
    print(result['messages'][-1].content)

✅ Escalation Agent (Ticket Creator) Ready.

--- Testing Escalation Agent Locally ---
✅ Support Ticket Created Successfully.\n🎫 **Ticket ID:** INC-1011\n📝 **Summary:** My VPN is totally broken and I need help now....\n\nA human specialist has been notified and will contact you shortly.
